## Thực hiện lưu từng dòng trong cột "Đáp án" thành 1 chunk trong Qdrant mà không cần chia nhỏ nữa,

### Import các thư viện cần thiết

In [ ]:
### Import các thư viện cần thiết
!pip install langchain qdrant-client -q

In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Qdrant
import pandas as pd

# Define constants for Qdrant and Hugging Face API keys
# QDRANT_API_KEY = "WbQ_"
# QDRANT_URL = ""
# EMBEDDINGS_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
# HUGGINGFACE_API_KEY = "hf_" # Your Hugging Face API key
# COLLECTION_NAME = ""


In [ ]:

# Load Excel data
DATA = 'LegalRAG.xlsx'
data = pd.read_excel(DATA)

# Extract and preprocess the "Đáp án" column
answers = data["Đáp án"].dropna().tolist()


In [ ]:

# Generate embeddings using HuggingFaceInferenceAPIEmbeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name=EMBEDDINGS_MODEL_NAME,
    api_key=HUGGINGFACE_API_KEY  # Correctly setting API key
)

# Create a vector database in Qdrant
qdrant = Qdrant.from_texts(
    texts=answers,  # Directly use each row as a chunk
    embedding=embeddings,
    url=QDRANT_URL,
    collection_name=COLLECTION_NAME,
    api_key=QDRANT_API_KEY,
    prefer_grpc=False,
)

# Verify if the data is added successfully
print(f"Number of chunks added: {len(answers)}")


## Add thêm sources

Để thêm thông tin metadata như `"source"` vào mỗi chunk và định dạng như bạn yêu cầu, chúng ta sẽ làm như sau:

1. **Thêm metadata:** Gắn thêm trường `source` chứa đường dẫn đến tệp dữ liệu gốc.
2. **Sử dụng vòng lặp:** Lặp qua các chunk và in ra nội dung cùng metadata như bạn yêu cầu.


In [ ]:
# from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
# from langchain.vectorstores import Qdrant
# from langchain.schema import Document  # Import Document class
# import pandas as pd

# # # Define constants for Qdrant and Hugging Face API keys
# QDRANT_API_KEY = "WbQ_"
# QDRANT_URL = ""
# EMBEDDINGS_MODEL_NAME = ""
# HUGGINGFACE_API_KEY = "hf_" # Your Hugging Face API key
# COLLECTION_NAME = ""

# # File source for metadata
# SOURCE_FILE = "LegalRAG.xlsx"

# # Load Excel data
# data = pd.read_excel(SOURCE_FILE)

# # Extract and preprocess the "Đáp án" column
# answers = data["Đáp án"].dropna().tolist()

# # Add metadata to each chunk and create Document objects
# documents = [Document(page_content=answer, metadata={"source": SOURCE_FILE}) for answer in answers]

# # Generate embeddings using HuggingFaceInferenceAPIEmbeddings
# embeddings = HuggingFaceInferenceAPIEmbeddings(
#     model_name=EMBEDDINGS_MODEL_NAME,
#     api_key=HUGGINGFACE_API_KEY
# )

# # Create a vector database in Qdrant
# qdrant = Qdrant.from_documents(
#     documents=documents,
#     embedding=embeddings,
#     url=QDRANT_URL,
#     collection_name=COLLECTION_NAME,
#     api_key=QDRANT_API_KEY,
#     prefer_grpc=False,
# )

# # Verify if the data is added successfully and print chunks
# for i, doc in enumerate(documents):
#     print(f"Document {i + 1} content: {doc.page_content[:100]}")  # In 100 ký tự đầu tiên
#     print(f"Document {i + 1} metadata: {doc.metadata}")
#     if not doc.page_content.strip():
#         print(f"Document {i + 1} is empty!")


### Output
Mỗi chunk sẽ có thông tin sau:
- **`content`**: Nội dung của chunk (các dòng từ cột "Đáp án").
- **`metadata`**: Metadata bao gồm thông tin về nguồn dữ liệu (`source`).

Ví dụ:
```plaintext
Document 1 content: Luật này quy định về quy tắc giao thông đường bộ; kết cấu hạ tầng giao thông đường bộ; phương tiện...
Document 1 metadata: {'source': 'LegalRAG.xlsx'}
```

### Giải thích
1. **`documents` array**: Chứa danh sách các đoạn văn bản với `page_content` và `metadata`.
2. **Thêm metadata `source`**: Gắn thông tin nguồn tệp vào metadata của mỗi chunk.
3. **In nội dung và metadata**: Kiểm tra từng chunk để đảm bảo dữ liệu đã được xử lý đúng cách.

Hãy thử chạy và kiểm tra kết quả!

## add thêm Question

In [ ]:
# Kiểm tra hàng có "Câu hỏi" nhưng không có "Đáp án"
questions_without_answers = data[data["Câu hỏi"].notna() & data["Đáp án"].isna()]

# Kiểm tra hàng có "Đáp án" nhưng không có "Câu hỏi"
answers_without_questions = data[data["Đáp án"].notna() & data["Câu hỏi"].isna()]

# Hiển thị kết quả
print("Rows with questions but no answers:")
print(questions_without_answers)

print("\nRows with answers but no questions:")
print(answers_without_questions)


In [ ]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.schema import Document  # Import Document class
import pandas as pd

QDRANT_API_KEY = "WbQ_"
QDRANT_URL = ""
EMBEDDINGS_MODEL_NAME = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
HUGGINGFACE_API_KEY = "hf_" # Your Hugging Face API key
COLLECTION_NAME = ""

# File source for metadata
SOURCE_FILE = "LegalRAG.xlsx"

# Load Excel data
data = pd.read_excel(SOURCE_FILE)

# Extract and preprocess the "Câu hỏi" and "Đáp án" columns
questions = data["Câu hỏi"].dropna().tolist()
answers = data["Đáp án"].dropna().tolist()

# Ensure questions and answers have the same length
assert len(questions) == len(answers), "Mismatch between number of questions and answers!"

# Add metadata to each chunk and create Document objects
documents = [
    Document(
        page_content=answer,
        metadata={"source": SOURCE_FILE, "question": question}
    )
    for question, answer in zip(questions, answers)
]

# Generate embeddings using HuggingFaceInferenceAPIEmbeddings
embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name=EMBEDDINGS_MODEL_NAME,
    api_key=HUGGINGFACE_API_KEY
)

# Create a vector database in Qdrant
qdrant = Qdrant.from_documents(
    documents=documents,
    embedding=embeddings,
    url=QDRANT_URL,
    collection_name=COLLECTION_NAME,
    api_key=QDRANT_API_KEY,
    prefer_grpc=False,
)

# Verify if the data is added successfully and print chunks
for i, doc in enumerate(documents):
    print(f"Document {i + 1} content: {doc.page_content[:100]}")  # In 100 ký tự đầu tiên
    print(f"Document {i + 1} metadata: {doc.metadata}")
    if not doc.page_content.strip():
        print(f"Document {i + 1} is empty!")



### Thay đổi chính
1. **Thêm metadata `question`:** Thêm trường `question` vào metadata cho mỗi chunk.
2. **Kiểm tra độ dài danh sách:** Đảm bảo rằng số lượng câu hỏi và câu trả lời khớp nhau để tránh lỗi khi tạo `Document`.
3. **Metadata đầy đủ:** Mỗi `Document` giờ sẽ có cả trường `source` và `question`.

### Output
Mỗi tài liệu sẽ có metadata như:
```plaintext
Document 1 content: Luật này quy định về quy tắc giao thông đường bộ...
Document 1 metadata: {'source': 'LegalRAG.xlsx', 'question': 'Trình bày khái niệm, chế độ pháp lý vùng nội thủy...'}
``` 

Hãy thử và kiểm tra kết quả! 😊